# Baseline Bus System

Implementation of 5 busses, conventional generator at bus 1, wind generation at bus 5, dynamic load at bus 2 and ordinary demands at busses 1, 3 and 4.
 
Line connections run from:
 
5-1;
1-2;
2-3;
2-4
 
 
Extend with actual data

In [49]:
using JuMP, HiGHS

# ------------------------------
# Data and Parameters
# ------------------------------

# Time periods
T = 1:8

# Scenarios: 9 scenarios (combining 3 wind scenarios and 3 load scenarios)
scenarios = 1:9

# Scenario probabilities (equal probability for each scenario)
prob = Dict(s => 1/9 for s in scenarios)

# Wind availability at Bus 5 (stochastic, per scenario and time period)
W = Dict(
    (1,1) => 32.63, (1,2) => 31.40, (1,3) => 28.52, (1,4) => 31.40, (1,5) => 33.26, (1,6) => 33.43, (1,7) => 34.59, (1,8) => 34.06,
    (2,1) => 32.63, (2,2) => 31.40, (2,3) => 28.52, (2,4) => 31.40, (2,5) => 33.26, (2,6) => 33.43, (2,7) => 34.59, (2,8) => 34.06,
    (3,1) => 32.63, (3,2) => 31.40, (3,3) => 28.52, (3,4) => 31.40, (3,5) => 33.26, (3,6) => 33.43, (3,7) => 34.59, (3,8) => 34.06,
    
    (4,1) => 15.79, (4,2) => 13.04, (4,3) => 10.11, (4,4) => 10.33, (4,5) => 11.47, (4,6) => 12.69, (4,7) => 15.69, (4,8) => 16.73,
    (5,1) => 15.79, (5,2) => 13.04, (5,3) => 10.11, (5,4) => 10.33, (5,5) => 11.47, (5,6) => 12.69, (5,7) => 15.69, (5,8) => 16.73,
    (6,1) => 15.79, (6,2) => 13.04, (6,3) => 10.11, (6,4) => 10.33, (6,5) => 11.47, (6,6) => 12.69, (6,7) => 15.69, (6,8) => 16.73,

    (7,1) => 53.79, (7,2) => 54.13, (7,3) => 59.43, (7,4) => 66.39, (7,5) => 67.63, (7,6) => 64.22, (7,7) => 60.45, (7,8) => 55.51,
    (8,1) => 53.79, (8,2) => 54.13, (8,3) => 59.43, (8,4) => 66.39, (8,5) => 67.63, (8,6) => 64.22, (8,7) => 60.45, (8,8) => 55.51,
    (9,1) => 53.79, (9,2) => 54.13, (9,3) => 59.43, (9,4) => 66.39, (9,5) => 67.63, (9,6) => 64.22, (9,7) => 60.45, (9,8) => 55.51
)

# Demand Profiles for Busses 1-4
D = Dict(
    (1,1) => 5.0, (1,2) => 8.2, (1,3) => 20.5, (1,4) => 35.0, (1,5) => 42.8, (1,6) => 38.4, (1,7) => 25.1, (1,8) => 12.0,
    (2,1) => 10.0, (2,2) => 15.4, (2,3) => 30.7, (2,4) => 50.2, (2,5) => 61.3, (2,6) => 55.0, (2,7) => 35.7, (2,8) => 18.5,
    (3,1) => 15.0, (3,2) => 22.5, (3,3) => 42.1, (3,4) => 68.9, (3,5) => 82.5, (3,6) => 74.2, (3,7) => 48.6, (3,8) => 25.9,

    (4,1) => 5.0, (4,2) => 8.2, (4,3) => 20.5, (4,4) => 35.0, (4,5) => 42.8, (4,6) => 38.4, (4,7) => 25.1, (4,8) => 12.0,
    (5,1) => 10.0, (5,2) => 15.4, (5,3) => 30.7, (5,4) => 50.2, (5,5) => 61.3, (5,6) => 55.0, (5,7) => 35.7, (5,8) => 18.5,
    (6,1) => 15.0, (6,2) => 22.5, (6,3) => 42.1, (6,4) => 68.9, (6,5) => 82.5, (6,6) => 74.2, (6,7) => 48.6, (6,8) => 25.9,

    (7,1) => 5.0, (7,2) => 8.2, (7,3) => 20.5, (7,4) => 35.0, (7,5) => 42.8, (7,6) => 38.4, (7,7) => 25.1, (7,8) => 12.0,
    (8,1) => 10.0, (8,2) => 15.4, (8,3) => 30.7, (8,4) => 50.2, (8,5) => 61.3, (8,6) => 55.0, (8,7) => 35.7, (8,8) => 18.5,
    (9,1) => 15.0, (9,2) => 22.5, (9,3) => 42.1, (9,4) => 68.9, (9,5) => 82.5, (9,6) => 74.2, (9,7) => 48.6, (9,8) => 25.9
)

# Conventional generation parameters at Bus 1
Gmax = 250.0   # Maximum generation capacity at Bus 1
c_g = 10.0     # Generation cost per unit
c_curt = 5.0 # Penalty cost per unit of wind curtailment 

# Penalty cost for load shedding 
c_ls = 100.0

# Ramping limits for conventional generation (per time period)
ramp_up = 50.0
ramp_down = 50.0

# Network parameters for DC load flow
X = 1.0  # Reactance 
Fmax = 250.0  # Flow limits on all lines

250.0

In [50]:
# ------------------------------
# Model Definition
# ------------------------------

model = Model(HiGHS.Optimizer)

# Conventional generation at Bus 1
@variable(model, g_conv[s in scenarios, t in T] >= 0, upper_bound = Gmax)

# Wind generation at Bus 5
@variable(model, g_wind[s in scenarios, t in T] >= 0)
# Wind curtailment at Bus 5
@variable(model, c_w[s in scenarios, t in T] >= 0)

for s in scenarios, t in T
    @constraint(model, g_wind[s,t] + c_w[s,t] == W[(s,t)])
end

# ------------------------------
# Network Flow Variables and Voltage Angles
# ------------------------------

# Network flows on each line  
@variable(model, f51[s in scenarios, t in T])
@variable(model, f12[s in scenarios, t in T])
@variable(model, f23[s in scenarios, t in T])
@variable(model, f24[s in scenarios, t in T])

# Voltage angles at buses 1 through 5  
@variable(model, theta[bus in 1:5, s in scenarios, t in T])
# Set reference angle at Bus 1 to 0 for all scenarios and time periods
for s in scenarios, t in T
    @constraint(model, theta[1,s,t] == 0)
end

# ------------------------------
# Load Shedding Variables at Buses 3 and 4
# ------------------------------

@variable(model, LS3[s in scenarios, t in T] >= 0)
@variable(model, LS4[s in scenarios, t in T] >= 0)

# ------------------------------
# Power Balance Constraints
# ------------------------------

# Bus 5 (Wind Bus): All wind generation flows out on the line 5-1.
for s in scenarios, t in T
    @constraint(model, g_wind[s,t] - f51[s,t] == 0)
end

# Bus 1 (Conventional Gen + Load D1): Generation plus inflow from Bus 5 must meet load D1 plus outflow.
for s in scenarios, t in T
    @constraint(model, g_conv[s,t] + f51[s,t] - f12[s,t] - D[(s,t)] == 0)
end

# Bus 2: The inflow from Bus 1 (f12) must supply load D2 and send out flows to Buses 3 and 4.
for s in scenarios, t in T
    @constraint(model, f12[s,t] - f23[s,t] - f24[s,t] - D[(s,t)] == 0)
end

# Bus 3: The inflow from Bus 2 plus load shedding LS3 must meet load D3.
for s in scenarios, t in T
    @constraint(model, f23[s,t] + LS3[s,t] - D[(s,t)] == 0)
end

# Bus 4: The inflow from Bus 2 plus load shedding LS4 must meet load D4.
for s in scenarios, t in T
    @constraint(model, f24[s,t] + LS4[s,t] - D[(s,t)] == 0)
end

# ------------------------------
# DC Power Flow Equations and Line Flow Limits
# ------------------------------

for s in scenarios, t in T
    @constraint(model, f51[s,t] == (theta[5,s,t] - theta[1,s,t]) / X)
    @constraint(model, f12[s,t] == (theta[1,s,t] - theta[2,s,t]) / X)
    @constraint(model, f23[s,t] == (theta[2,s,t] - theta[3,s,t]) / X)
    @constraint(model, f24[s,t] == (theta[2,s,t] - theta[4,s,t]) / X)
    
    @constraint(model, f51[s,t] <= Fmax)
    @constraint(model, -f51[s,t] <= Fmax)
    @constraint(model, f12[s,t] <= Fmax)
    @constraint(model, -f12[s,t] <= Fmax)
    @constraint(model, f23[s,t] <= Fmax)
    @constraint(model, -f23[s,t] <= Fmax)
    @constraint(model, f24[s,t] <= Fmax)
    @constraint(model, -f24[s,t] <= Fmax)
end

# ------------------------------
# Ramping Constraints for Conventional Generation at Bus 1
# ------------------------------

for s in scenarios, t in T[2:end]
    @constraint(model, g_conv[s,t] - g_conv[s,t-1] <= ramp_up)
    @constraint(model, g_conv[s,t-1] - g_conv[s,t] <= ramp_down)
end

# ------------------------------
# Objective Function
# ------------------------------
@objective(model, Min, 
    sum(prob[s] * (c_g * g_conv[s,t] + c_curt * c_w[s,t] + c_ls * (LS3[s,t] + LS4[s,t]))
        for s in scenarios, t in T)
)


1.1111111111111112 g_conv[1,1] + 0.5555555555555556 c_w[1,1] + 11.11111111111111 LS3[1,1] + 11.11111111111111 LS4[1,1] + 1.1111111111111112 g_conv[1,2] + 0.5555555555555556 c_w[1,2] + 11.11111111111111 LS3[1,2] + 11.11111111111111 LS4[1,2] + 1.1111111111111112 g_conv[1,3] + 0.5555555555555556 c_w[1,3] + 11.11111111111111 LS3[1,3] + 11.11111111111111 LS4[1,3] + 1.1111111111111112 g_conv[1,4] + 0.5555555555555556 c_w[1,4] + 11.11111111111111 LS3[1,4] + 11.11111111111111 LS4[1,4] + 1.1111111111111112 g_conv[1,5] + 0.5555555555555556 c_w[1,5] + 11.11111111111111 LS3[1,5] + 11.11111111111111 LS4[1,5] + 1.1111111111111112 g_conv[1,6] + 0.5555555555555556 c_w[1,6] + 11.11111111111111 LS3[1,6] + 11.11111111111111 LS4[1,6] + 1.1111111111111112 g_conv[1,7] + 0.5555555555555556 c_w[1,7] + 11.11111111111111 LS3[1,7] + 11.11111111111111 LS4[1,7] + 1.1111111111111112 g_conv[1,8] + 0.5555555555555556 c_w[1,8] + [[...228 terms omitted...]] + 11.11111111111111 LS3[9,1] + 11.11111111111111 LS4[9,1] + 1.

In [51]:
# ------------------------------
# Solving / Model Analyisis
# ------------------------------
optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [6e-01, 1e+01]
  Bound  [2e+02, 2e+02]
  RHS    [5e+00, 2e+02]
Presolving model
270 rows, 360 cols, 684 nonzeros  0s
198 rows, 288 cols, 540 nonzeros  0s
113 rows, 176 cols, 280 nonzeros  0s
113 rows, 176 cols, 280 nonzeros  0s
Presolve : Reductions: rows 113(-1381); columns 176(-832); elements 280(-2492)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         95     1.5918283333e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 95
Objective value     :  1.5918283333e+04
Relative P-D gap    :  1.8283272038e-15
HiGHS run time      :          0.01


In [ ]:
println("Overall Optimal Objective Value: ", objective_value(model))

# Print results per scenario and time period
for s in scenarios, t in T
    println("Scenario $s, Time Period $t:")
    println("  Conventional Generation (Bus 1): ", value(g_conv[s,t]))
    println("  Wind Generation Used (Bus 5): ", value(g_wind[s,t]))
    println("  Wind Curtailment (Bus 5): ", value(c_w[s,t]))
    println("  Load at each Bus: ", D[(s,t)])
    println("  Load Shedding (Bus 3): ", value(LS3[s,t]))
    println("  Load Shedding (Bus 4): ", value(LS4[s,t]))
end

Overall Optimal Objective Value: 15918.283333333331
Scenario 1, Time Period 1:
  Conventional Generation (Bus 1): 0.0
  Wind Generation Used (Bus 5): 20.0
  Wind Curtailment (Bus 5): 12.630000000000003
  Load at each Bus: 5.0
  Load Shedding (Bus 3): 0.0
  Load Shedding (Bus 4): 0.0
Scenario 1, Time Period 2:
  Conventional Generation (Bus 1): 8.599999999999994
  Wind Generation Used (Bus 5): 24.200000000000003
  Wind Curtailment (Bus 5): 7.199999999999996
  Load at each Bus: 8.2
  Load Shedding (Bus 3): 0.0
  Load Shedding (Bus 4): 0.0
Scenario 1, Time Period 3:
  Conventional Generation (Bus 1): 58.599999999999994
  Wind Generation Used (Bus 5): 23.400000000000006
  Wind Curtailment (Bus 5): 5.119999999999994
  Load at each Bus: 20.5
  Load Shedding (Bus 3): 0.0
  Load Shedding (Bus 4): 0.0
Scenario 1, Time Period 4:
  Conventional Generation (Bus 1): 108.6
  Wind Generation Used (Bus 5): 31.4
  Wind Curtailment (Bus 5): 0.0
  Load at each Bus: 35.0
  Load Shedding (Bus 3): 0.0
  Loa

In [53]:
# print the sum of conventional generation over all scenarios and time periods, sum of curtailment and sum of load shedding
println("Expected Conventional Generation: ", sum(prob[s] * sum(value(g_conv[s,t]) for t in T) for s in scenarios))
println("Expected Wind Curtailment: ", sum(prob[s] * sum(value(c_w[s,t]) for t in T) for s in scenarios))
println("Expected Load Shedding: ", sum(prob[s] * sum(value(LS3[s,t] + LS4[s,t]) for t in T) for s in scenarios))

Expected Conventional Generation: 848.601111111111
Expected Wind Curtailment: 76.65444444444444
Expected Load Shedding: 70.49000000000001
